Leung Wai Liu <br>
JPMC-SMM4H <br>
July 17, 2022 <br>
Task 2b DEV Weighted Logits Average Ensembling

In [32]:
import pandas as pd
import numpy as np
from collections import Counter
from labels_to_ids import task7_labels_to_ids
from sklearn.metrics import accuracy_score, classification_report, f1_score, precision_score, recall_score, confusion_matrix
from training_code import calculate_overall_performance_metrics
import os
from scipy.special import softmax


In [33]:
# Loading up all the predictions data

n_rnds = 5
original_df = pd.read_csv('../Datasets/dev.tsv', sep='\t')
# models = ['bert-large-uncased']
# models = ['roberta-large']
models = ['bert-large-uncased', 'roberta-large']
n_models = len(models)

epoch_string = '../15_epochs_large_model/eval_testing/saved_eval_test_result_2b'
eval_logits_string = '../15_epochs_large_model/eval_testing/saved_eval_report_2b'
n_rows = len(original_df)

labels_to_ids = task7_labels_to_ids
ids_to_labels = dict((v,k) for k,v in labels_to_ids.items())

# Loading up all of the results
best_f1 = pd.read_csv('../15_epochs_large_model/eval_testing/validation_stats/all_best_overall_f1_score.tsv', sep='\t')
best_f1



,Unnamed: 0,bert-large-uncased,roberta-large
0,0,0.788181,0.805198
1,1,0.798593,0.789957
2,2,0.771915,0.777964
3,3,0.813640,0.796570
4,4,0.784882,0.782647


In [34]:
# Retrieving all the predictions from the 
list_of_df = []
list_of_logits = []
list_of_f1_score = []
sum_of_all_f1_score = 0

for model in models: 
    specific_model_row = []
    specific_model_row_logits = []
    specific_model_row_f1 = []

    for rnd in range(n_rnds):
        to_read_string = epoch_string + '/' + model + '/' + str(rnd) + '/unformatted_eval_test_result.tsv'
        to_read_eval_string = eval_logits_string + '/' + model + '/' + str(rnd) + '/eval_logits.tsv'

        # Reading the best f1 scores 
        specific_f1_score = best_f1.at[rnd, model]
        specific_model_row_f1.append(specific_f1_score)
        sum_of_all_f1_score += specific_f1_score
        
        # Reading each particular model df
        particular_model_df = pd.read_csv(to_read_string, sep='\t')
        specific_model_row.append(particular_model_df)

        # Reading each particular model's eval logits
        particular_model_eval_logits = pd.read_csv(to_read_eval_string, sep='\t')
        particular_model_eval_logits = particular_model_eval_logits.drop(columns = 'Unnamed: 0')
        particular_model_eval_logits = list(particular_model_eval_logits.itertuples(index=False, name=None))
        particular_model_eval_logits = softmax(particular_model_eval_logits, axis = 1)
        # particular_model_eval_logits = softmax(particular_model_eval_logits)
        specific_model_row_logits.append(particular_model_eval_logits)
    
    list_of_df.append(specific_model_row)
    list_of_logits.append(specific_model_row_logits)
    list_of_f1_score.append(specific_model_row_f1)


In [35]:
print(list_of_logits)

[[array([[0.89585086, 0.10414914],
       [0.99063968, 0.00936032],
       [0.42433155, 0.57566845],
       ...,
       [0.22545724, 0.77454276],
       [0.48912451, 0.51087549],
       [0.85860647, 0.14139353]]), array([[0.99021053, 0.00978947],
       [0.99626552, 0.00373448],
       [0.34279566, 0.65720434],
       ...,
       [0.0689863 , 0.9310137 ],
       [0.58620965, 0.41379035],
       [0.99465936, 0.00534064]]), array([[0.73402802, 0.26597198],
       [0.97611812, 0.02388188],
       [0.34621537, 0.65378463],
       ...,
       [0.2992464 , 0.7007536 ],
       [0.52008634, 0.47991366],
       [0.90895835, 0.09104165]]), array([[0.82101694, 0.17898306],
       [0.98860378, 0.01139622],
       [0.52075668, 0.47924332],
       ...,
       [0.23455309, 0.76544691],
       [0.6195134 , 0.3804866 ],
       [0.87333453, 0.12666547]]), array([[0.77456963, 0.22543037],
       [0.98617556, 0.01382444],
       [0.7103671 , 0.2896329 ],
       ...,
       [0.2124824 , 0.7875176 ],
      

In [36]:
list_of_f1_score

[[0.7881813080614757,
  0.7985929620825077,
  0.771914781929742,
  0.8136395698230106,
  0.7848816733870172],
 [0.8051984112145483,
  0.7899571062545162,
  0.7779635632956179,
  0.7965695351611923,
  0.7826465572751184]]

In [37]:
# TAKING THE WEIGHTED AVERAGE OF DATA

weighted_avg_original_tweet_id_list = []
weighted_avg_original_sentence_list = []
weighted_avg_original_claim_list = []
weighted_avg_original_label_list = []
weighted_avg_original_numbered_label_list = []

weighted_avg_predicted_number_results = []
weighted_avg_predicted_results = []


for index, row in original_df.iterrows(): 
    # getting the original values in the tweet
    original_tweet_id = row['id']
    original_sentence = row['Tweet']
    original_claim = row['Claim']
    original_label = row['Premise']
    
    # transferring the labels over to final list
    weighted_avg_original_tweet_id_list.append(original_tweet_id)
    weighted_avg_original_sentence_list.append(original_sentence)
    weighted_avg_original_claim_list.append(original_claim)
    weighted_avg_original_label_list.append(original_label)
    
    specific_row_value = 0.0
    # go through every models' row of data 
    
    for model_num in range(n_models):
        for rnd_num in range(n_rnds):

            row_logits = list_of_logits[model_num][rnd_num][index]

            row_logits_value = ((row_logits[0] * 0) + (row_logits[1] * 1))
            prediction = row_logits_value * list_of_f1_score[model_num][rnd_num]
            print(prediction)
            specific_row_value += prediction

    specific_row_value = specific_row_value / sum_of_all_f1_score
    specific_row_result = int(round(specific_row_value))
    weighted_avg_predicted_results.append(specific_row_result)
    

0.08208840805779212
0.007817801614319515
0.20530770265694187
0.14562770369054737
0.17693616953908006
0.014159737188753024
0.0001226450004408714
0.0009893419400842222
0.0006498595830041886
0.06404033428404175
0.007377630745878224
0.002982326450506503
0.018434779550610254
0.00927241483191887
0.01085054854351592
0.009770614409732705
0.00013723475430344817
0.0019757699386576165
0.0003060934854595501
0.002596986115389533
0.45373111303553953
0.5248387596560983
0.5046660209948268
0.3899313315347277
0.22732755466993768
0.39553888822664424
0.76919593584595
0.7666763147995497
0.0052113189584745605
0.43343565875483586
0.18312349513603401
0.3369262359011594
0.07654300655237777
0.19821675239513903
0.10435567271153001
0.01058967347100863
0.00018256844694862408
0.002370623329963876
0.0014305152772621445
0.023131105322913976
0.2190156809585318
0.18830590219215712
0.31275271728443627
0.35812401699494606
0.3302197876066402
0.3927797472378091
0.04266801493093179
0.7750608724883264
0.2609814080936481
0.03

In [38]:
# Calculating sklearn metrics

num_overall_prediction_data = pd.DataFrame(zip(weighted_avg_original_tweet_id_list, weighted_avg_original_sentence_list, weighted_avg_original_claim_list, weighted_avg_original_label_list, weighted_avg_predicted_results), columns=['tweet_id', 'text', 'Claim', 'Orig', 'Premise'])

fm_f1_score, fm_precision, fm_recall, saho_f1_score, saho_precision, saho_recall, sc_f1_score, sc_precision, sc_recall = calculate_overall_performance_metrics(num_overall_prediction_data)

accuracy_result = accuracy_score(weighted_avg_original_label_list, weighted_avg_predicted_results)

net_f1 = (1.0/3.0) * (fm_f1_score + saho_f1_score + sc_f1_score)

print("ACCURACY:", accuracy_result)
print("F1:", net_f1)

# Saving results to file
os.makedirs('../15_epochs_large_model/eval_testing/eval_validation_statistics/weighted_logits_avg_ensemble', exist_ok=True)

with open('../15_epochs_large_model/eval_testing/eval_validation_statistics/weighted_avg_ensemble/weighted_logits_avg_ensemble_valid_stats.txt', 'w') as file:
        file.write("Accuracy: " + str(accuracy_result) + "\n")
        file.write("Net F1: " + str(net_f1) + "\n")
        file.write("Ind F1 Score: " + str(fm_f1_score) + " , " + str(saho_f1_score) + " , " + str(sc_f1_score) + "\n")
        file.write("Ind Precision Score: " + str(fm_precision) + " , " + str(saho_precision) + " , " + str(sc_precision) + "\n")
        file.write("Ind Recall Score: " + str(fm_recall) + " , " + str(saho_recall) + " , " + str(sc_recall) + "\n")


Running performance metrics
Finished running performance metrics
ACCURACY: 0.8333333333333334
F1: 0.8175959769249606
